In [1]:
import requests 
import pandas as pd

/var/folders/m5/qxcyvtcn2bd1qt6xxnplw0pw0000gn/T/ipykernel_12190/4199147742.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
import webbrowser

client_id = '36dd0c84bf9f436998ef368e1bcd46de'
client_secret = 'aa1120d6efcf49c4987dce0c3c1dacfb'
redirect_uri = 'https://developer.spotify.com/'
scope = 'user-top-read'

auth_url = f'https://accounts.spotify.com/authorize?response_type=code&client_id={client_id}&scope={scope}&redirect_uri={redirect_uri}'
webbrowser.open(auth_url)


True

In [15]:
import requests

# Replace with the authorization code you received
authorization_code = input('Enter the authorization code from the URL: ')

token_url = 'https://accounts.spotify.com/api/token'
client_id = '36dd0c84bf9f436998ef368e1bcd46de'
client_secret = 'aa1120d6efcf49c4987dce0c3c1dacfb'
redirect_uri = 'https://developer.spotify.com/'

response = requests.post(token_url, data={
    'grant_type': 'authorization_code',
    'code': authorization_code,
    'redirect_uri': redirect_uri,
    'client_id': client_id,
    'client_secret': client_secret,
})

# Check if the request was successful
if response.status_code != 200:
    print('Error getting access token:', response.json())
else:
    token_data = response.json()
    if 'access_token' in token_data:
        access_token = token_data['access_token']
        print('Access token obtained successfully')
    else:
        print('Access token not found in response:', token_data)



Access token obtained successfully


In [16]:
def fetch_top_tracks(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    endpoint_url = 'https://api.spotify.com/v1/me/top/tracks'
    response = requests.get(endpoint_url, headers=headers)
    
    if response.status_code != 200:
        print('Error fetching data:', response.json())
        return None
    
    data = response.json()
    
    if 'items' not in data:
        print('No items found in the response:', data)
        return None
    
    tracks = []
    for item in data['items']:
        tracks.append({
            'Track Name': item['name'],
            'Artist': item['artists'][0]['name'],
            'Popularity': item['popularity'],
            'Album': item['album']['name']
        })
    
    return tracks

# Fetch and print the top tracks
top_tracks = fetch_top_tracks(access_token)
if top_tracks:
    for track in top_tracks:
        print(f"Track Name: {track['Track Name']}, Artist: {track['Artist']}, Popularity: {track['Popularity']}, Album: {track['Album']}")


Track Name: Box Fan Slumber, Artist: Dozy Sounds, Popularity: 51, Album: Dozy Air
Track Name: Creatures in Heaven, Artist: Glass Animals, Popularity: 74, Album: Creatures In Heaven
Track Name: FE!N (feat. Playboi Carti), Artist: Travis Scott, Popularity: 92, Album: UTOPIA
Track Name: De Selby (Part 2), Artist: Hozier, Popularity: 65, Album: Unreal Unearth
Track Name: Got Me Started, Artist: Troye Sivan, Popularity: 76, Album: Something To Give Each Other
Track Name: Jackie and Wilson, Artist: Hozier, Popularity: 72, Album: Hozier (Expanded Edition)
Track Name: Dance Now (feat. Kenny Mason), Artist: JID, Popularity: 68, Album: The Forever Story (Extended Version)
Track Name: Damage Gets Done (feat. Brandi Carlile), Artist: Hozier, Popularity: 62, Album: Unreal Unearth
Track Name: Almost (Sweet Music), Artist: Hozier, Popularity: 75, Album: Wasteland, Baby!
Track Name: THE REV3NGE, Artist: Joey Bada$$, Popularity: 60, Album: THE REV3NGE
Track Name: Waving Through A Window, Artist: Ben Pl

In [19]:
# Function to get top artists for a specific time range
def fetch_top_artists(access_token, time_range='medium_term', limit=50):
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    endpoint_url = f'https://api.spotify.com/v1/me/top/artists?time_range={time_range}&limit={limit}'
    response = requests.get(endpoint_url, headers=headers)
    
    if response.status_code != 200:
        print('Error fetching data:', response.json())
        return None
    
    data = response.json()
    artists = []
    for item in data['items']:
        artists.append({
            'Artist Name': item['name'],
            'Artist ID': item['id'],
            'Popularity': item['popularity'],
            'Genres': item['genres'],
            'Time Range': time_range
        })
    
    return pd.DataFrame(artists)

# Assuming access_token is already obtained
access_token = 'AQBUgV4VtbfLVfRQyFJREXLwGCEKwBKzi2VpfCTPFmlT7vJRFGM-31MV4i4HTK9JWvJlIuQ8Eq9Vfb90GqmEuptHN1EHsaBtEKgY1ddiNxUCqL54IHS-9iHmQRq1DAuChxphuXzlTFiasObg2KexwJ4TLIVQ_IaaMYfVEE7mOK-nW36XRIqpllCuz_WENeo_IQ'

# Fetch top artists for different time ranges
short_term_artists = fetch_top_artists(access_token, time_range='short_term')
medium_term_artists = fetch_top_artists(access_token, time_range='medium_term')
long_term_artists = fetch_top_artists(access_token, time_range='long_term')

# Combine the data into a single DataFrame
all_artists = pd.concat([short_term_artists, medium_term_artists, long_term_artists])

# Save to a CSV file
all_artists.to_csv('top_artists_over_time.csv', index=False)

print("Data fetched and saved to 'top_artists_over_time.csv'")


Error fetching data: {'error': {'status': 401, 'message': 'Invalid access token'}}
Error fetching data: {'error': {'status': 401, 'message': 'Invalid access token'}}
Error fetching data: {'error': {'status': 401, 'message': 'Invalid access token'}}


ValueError: All objects passed were None